In [1]:
import cv2
import os

# Initialize webcam
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Create dataset folder
dataset_path = "dataset"
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

# Ask for user name
user_name = input("Enter your name: ")
user_folder = os.path.join(dataset_path, user_name)

if not os.path.exists(user_folder):
    os.makedirs(user_folder)

print("Capturing images... Press 'q' to quit.")

count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        count += 1
        face_img = gray[y:y+h, x:x+w]
        cv2.imwrite(f"{user_folder}/{count}.jpg", face_img)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("Capturing Faces", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q') or count >= 100:
        break

cap.release()
cv2.destroyAllWindows()
print(f"Captured {count} images for {user_name}.")

2025-03-30 06:21:08.092 Python[41663:659990] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Capturing images... Press 'q' to quit.


2025-03-30 06:21:18.212 Python[41663:659990] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-30 06:21:18.212 Python[41663:659990] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Captured 100 images for Neev Gupta.


In [2]:
import cv2
import os
import numpy as np
import pickle

# Ensure OpenCV contrib module is installed
if not hasattr(cv2.face, "LBPHFaceRecognizer_create"):
    raise ImportError("OpenCV face module is missing! Install with: pip install opencv-contrib-python")

# Initialize the Face Recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Path to dataset folder
dataset_path = "dataset"
faces = []
labels = []
label_dict = {}
current_id = 0

# Process each person's folder in the dataset
for person_name in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person_name)
    if not os.path.isdir(person_path):
        continue

    # Assign an ID to each person
    if person_name not in label_dict:
        label_dict[person_name] = current_id
        current_id += 1

    person_id = label_dict[person_name]

    # Process each image in the person's folder
    for image_name in os.listdir(person_path):
        image_path = os.path.join(person_path, image_name)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            print(f"Skipping invalid image: {image_path}")
            continue

        faces.append(img)
        labels.append(person_id)

# Convert lists to numpy arrays
faces = np.array(faces, dtype="object")
labels = np.array(labels)

# Ensure training data is not empty
if len(faces) == 0:
    raise ValueError("Error: No valid face images found for training!")

print(f"Training on {len(faces)} images from {len(label_dict)} people...")

# Train the recognizer
recognizer.train(faces, labels)

# Save the trained model
recognizer.save("face_model.yml")

# Save label dictionary
with open("labels.pkl", "wb") as f:
    pickle.dump(label_dict, f)

print("Training complete! Model and labels saved.")

Training on 601 images from 6 people...
Training complete! Model and labels saved.


In [3]:
import cv2
import pickle
import numpy as np
from deepface import DeepFace
import time
from collections import Counter
# Load trained face recognition model
recognizer = cv2.face.LBPHFaceRecognizer_create()
try:
    recognizer.read("face_model.yml")
    print("Face model loaded successfully!")
except Exception as e:
    print("Error loading face model:", str(e))
    exit()

# Load label dictionary
try:
    with open("labels.pkl", "rb") as f:
        label_dict = pickle.load(f)
    print("Label dictionary loaded:", label_dict)
except Exception as e:
    print("Error loading labels:", str(e))
    exit()

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Start webcam
cap = cv2.VideoCapture(0)
start_time = time.time()  # Record start time
DURATION = 10  # Run for 10 seconds
emotion_records = []
while time.time() - start_time < DURATION:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not capture frame!")
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(50, 50))

    for (x, y, w, h) in faces:
        face_img = gray[y:y+h, x:x+w]  # Extract face region

        # Recognize the face
        label_id, confidence = recognizer.predict(face_img)

        # Dynamically adjust threshold
        ADAPTIVE_THRESHOLD = 60
        name = "Unknown"
        if label_id in label_dict.values() and confidence < ADAPTIVE_THRESHOLD:
            name = [k for k, v in label_dict.items() if v == label_id][0]

        # Detect emotions
        try:
            emotion_analysis = DeepFace.analyze(frame[y:y+h, x:x+w], actions=['emotion'], enforce_detection=False)
            detected_emotion = emotion_analysis[0]['dominant_emotion']
            emotion_records.append(detected_emotion) 
        except Exception as e:
            print("Emotion Detection Error:", str(e))
            detected_emotion = "Unknown"

        # Draw rectangle & text
        # Define text
        window_height, window_width, _ = frame.shape
        text = "Person: "+name+" | Emotion: "+detected_emotion
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 2
        font_thickness = 4
        text_color = (0, 102, 204) 

        # Get text size for positioning
        (text_width, text_height), _ = cv2.getTextSize(text, font, font_scale, font_thickness)

        # Position text at the **bottom center** of the window
        text_x = (window_width - text_width) // 2  # Center horizontally
        text_y = window_height - 50  # 20 pixels from the bottom

        # Draw text (separate from the face detection logic)
        cv2.putText(frame, text, (text_x, text_y), font, font_scale, text_color, font_thickness)

        color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        # cv2.putText(frame, f"{name} ({emotion})", (x, y - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"{name}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        

    # Display the frame
    cv2.imshow("Face & Emotion Recognition", frame)

    # Get the most frequent emotion
    most_frequent_emotion = Counter(emotion_records).most_common(1)[0][0] if emotion_records else "Unknown"


    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
print(f"Final Detected Emotion (after 10 seconds): {most_frequent_emotion}")
# Cleanup
cap.release()
cv2.waitKey(1) 
cv2.destroyAllWindows()
cv2.waitKey(1) 
cv2.destroyAllWindows()

Face model loaded successfully!
Label dictionary loaded: {'Sannidhya': 0, 'Neev Gupta': 1, 'Aakarah': 2, 'Aakarsh': 3, 'arpita': 4, 'Mayank': 5}
Final Detected Emotion (after 10 seconds): neutral


In [4]:
import pandas as pd
df=pd.read_csv("math_questions.csv")
mapping={"happy":"Hard","sad":"Medium","angry":"Easy","fear":"Easy",
           "surprise":"Hard","neutral":"Medium","disgust":"Medium"}
def get_quiz(detected_emotion, num_questions=10):
    difficulty = mapping.get(detected_emotion, "Medium")
    filt_df = df[df["Difficulty"] == difficulty]  
    if len(filt_df) < num_questions:
        extra_questions = df.sample(n=num_questions - len(filt_df), replace=True, random_state=None)
        selected_questions = pd.concat([filt_df, extra_questions]).reset_index(drop=True)
    else:
        selected_questions = filt_df.sample(n=num_questions, random_state=None).reset_index(drop=True)
    selected_questions.index += 1  
    return selected_questions
quiz_ques = get_quiz(detected_emotion)
print("{} difficulty questions based on your mood".format(mapping[detected_emotion]))
print(quiz_ques[["Question"]])

Easy difficulty questions based on your mood
                          Question
1                 What is 49 + 82?
2                 What is 25 + 44?
3   What is the square root of 73?
4                 What is 82 * 33?
5              What is 89 squared?
6                 What is 36 * 24?
7                 What is 51 * 99?
8                 What is 27 + 56?
9              What is 10 squared?
10   What is the square root of 1?
